In [90]:
import numpy as np
import math
import pandas as pd
import itertools
import copy
from sklearn.cluster import DBSCAN

In [161]:
from random import choice

In [2]:
np.random.seed(42)

# Constants and Data Loading

In [46]:
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

In [69]:
max_number_of_tracks = 232
max_number_of_tracks_power_2 = 256
max_number_of_tracks_log_2 = 8
batch_size = 50
eps = 0.15

z0_file = "/media/lucas/QS/binaries-trk/OldKF_TTbar_170K_quality-1-trk-z0.bin"
pt_file = "/media/lucas/QS/binaries-trk/OldKF_TTbar_170K_quality-1-trk-pt.bin"
z0 = np.fromfile(z0_file, dtype=np.float32)
pt = np.fromfile(pt_file, dtype=np.float32)

In [266]:
class BatchedDBSCAN:
    def __init__(
        self, z0, pt, eps, batch_size, max_number_of_tracks, verbose: bool = False, save_intermediate:bool =False
    ):

        self.eps = eps
        self.batch_size = batch_size
        self.verbose = verbose
        self.save_intermediate = save_intermediate
        self.z0_boundary = 21  # 21 cm is outside the detector acceptance
        self.pt_boundary = 0  # 0 pT won't contribute to the pT sum.
        self.minPts = 2  # This algorithm only works for a minimum number of 2 points

        self.max_number_of_tracks = int(max_number_of_tracks)
        self.n_batches = math.ceil(self.max_number_of_tracks / self.batch_size)

        # Max number of tracks including all batches
        self.max_n_tracks_batched = self.batch_size * self.n_batches
        self.max_n_clusters_batch = math.ceil(self.batch_size / self.minPts)
        self.max_n_clusters = math.ceil(self.max_n_tracks_batched / self.minPts)

        # Need to pad vectors to the max_number_of_tracks allowed so that it matches the fpga input
        n_pad = self.max_number_of_tracks - z0.shape[0]
        # if verbose:
        # print("original number of tracks: ", z0.shape)
        self.z0 = self.pad_vector(z0, n_pad, self.z0_boundary)
        self.pt = self.pad_vector(pt, n_pad, self.pt_boundary)

        # These are needed for the prefix sum
        self.max_number_of_tracks_power_2 = (
            1 << (self.max_number_of_tracks - 1).bit_length()
        )
        self.batch_size_power_2 = 1 << (self.batch_size - 1).bit_length()
        self.max_number_of_tracks_log_2 = np.log2(self.max_number_of_tracks_power_2)
        self.batch_size_log_2 = np.log2(self.batch_size_power_2)
        # self.n_batches = math.ceil(self.max_number_of_tracks / self.batch_size)

    def pad_vector(self, vec, n_pad, value):
        """pads vector to a set size with given value"""

        vec_to_pad = value * np.ones(n_pad)
        vec = np.append(vec, vec_to_pad)

        return vec

    def build_tracks(self, z0, pt):
        """Builds tracks batchess"""

        # Shape is determined by the size of batch, z0, pT and label (not used atm)
        track_batch = np.zeros((self.batch_size, 3))

        track_batch[:, 0] = z0
        track_batch[:, 1] = pt

        # sort the tracks by z0
        track_batch = track_batch[track_batch[:, 0].argsort()]

        return track_batch

    def prefix_sum(self, arr):
        """
        Calculates the prefix sum of pT.
        Warning, requires array to be of size thats log base of 2.
        """
        size_log2 = int(np.log2(arr.shape[0]))

        # up-sweep
        for d in range(0, size_log2, 1):
            step_size = 2**d
            double_step_size = step_size * 2

            for i in range(0, arr.shape[0], double_step_size):
                arr[i + double_step_size - 1] += arr[i + step_size - 1]

        # down-sweep
        arr[arr.shape[0] - 1] = 0
        d = size_log2 - 1

        while d >= 0:
            step_size = 2**d
            double_step_size = step_size * 2
            for i in range(0, arr.shape[0], double_step_size):
                tmp = arr[i + step_size - 1]
                arr[i + step_size - 1] = arr[i + double_step_size - 1]
                arr[i + double_step_size - 1] += tmp
            d -= 1

        return arr

    def find_left_boundaries(self, tracks):

        left_boundaries = np.zeros(self.batch_size, dtype=bool)

        # first value is always a left boundary
        left_boundaries[0] = 1

        for i in range(1, self.batch_size):
            _t = tracks[i]

            if _t[0] - tracks[i - 1][0] > self.eps:
                tracks[i][2] = -1
                left_boundaries[i] = 1
            else:
                left_boundaries[i] = 0

        return left_boundaries

    def find_right_boundaries(self, left_boundaries, rs, tracks):

        max_tracks = self.batch_size

        boundaries = np.zeros((max_tracks, 6))

        for i in range(max_tracks - 1):

            check1 = left_boundaries[i] and not (left_boundaries[i + 1])
            check2 = not (left_boundaries[i]) and left_boundaries[i + 1]

            if check1 or check2:
                boundaries[i][0] = i
                boundaries[i][1] = rs[i]
                boundaries[i][2] = rs[i + 1]
                boundaries[i][3] = rs[i + 1] - rs[i]
                boundaries[i][4] = tracks[i, 0]
                boundaries[i][5] = tracks[i + 1, 0]
            else:
                boundaries[i][0] = max_tracks
                boundaries[i][1] = 0
                boundaries[i][2] = 0
                boundaries[i][3] = 0
                boundaries[i][4] = 21
                boundaries[i][5] = 21

        # Check for the last boundary
        if left_boundaries[max_tracks - 1]:
            boundaries[max_tracks - 1][0] = max_tracks
            boundaries[max_tracks - 1][1] = 0
            boundaries[max_tracks - 1][2] = 0
            boundaries[max_tracks - 1][3] = 0
            boundaries[max_tracks - 1][4] = 21
            boundaries[max_tracks - 1][5] = 21
        else:
            boundaries[max_tracks - 1][0] = max_tracks - 1
            boundaries[max_tracks - 1][1] = rs[max_tracks - 1]
            boundaries[max_tracks - 1][2] = rs[max_tracks]
            boundaries[max_tracks - 1][3] = rs[max_tracks] - rs[max_tracks - 1]
            boundaries[max_tracks - 1][4] = tracks[max_tracks - 1, 0]
            boundaries[max_tracks - 1][5] = tracks[max_tracks - 1, 0]

        # Sort boundaries by the index
        boundaries = boundaries[boundaries[:, 0].argsort()]

        # # Add sum of Pt information
        # boundaries[:, 3] = boundaries[:, 2] - boundaries[:,  1]

        return boundaries

    def convert_boundaries_to_clusters(self, boundaries: np.array) -> np.array:
        n_boundaries = boundaries.shape[0]
        n_clusters = math.ceil(n_boundaries / 2)
        clusters = np.zeros((n_clusters, 6))
        j = 0
        for i in range(0, n_boundaries, 2):
            pt_low = boundaries[i, 1]
            pt_high = boundaries[i + 1, 2]
            pt_sum = pt_high - pt_low
            z0_low = boundaries[i, 4]
            z0_high = boundaries[i + 1, 5]

            clusters[j, 3] = pt_sum
            clusters[j, 4] = z0_low
            clusters[j, 5] = z0_high
            j += 1
        return clusters

    def get_vertex(self, cluster_of_tracks: np.array) -> float:
        """
        Calculates the median z0 of the cluster of tracks
        """

        n_size = cluster_of_tracks.shape[0]

        if n_size % 2 == 0:
            return 0.5 * (
                cluster_of_tracks[n_size // 2] + cluster_of_tracks[n_size // 2 - 1]
            )
        else:
            return cluster_of_tracks[n_size // 2]
        
    def merge_clusters(self, clusters:np.array) -> np.array: 
        
        n_clusters = clusters.shape[0]
        if self.n_batches == 1:
            self.max_pt_i = np.argmax(clusters[:, 3])
            self.max_pt = clusters[self.max_pt_i, 3]
            
            
        else:
            max_pt = 0
            max_pt_i = 0
            merge_count = 0
            
            comb = list(itertools.combinations(range(n_clusters), 2))
                        
            for i, j in comb:
                if clusters[i, 4] >= 21:
                    continue
                
                if max_pt < clusters[i, 3]:
                    max_pt = clusters[i,3]
                    max_pt_i = i
                
                if clusters[j, 4] >= 21:
                    continue
                
                case1 = (clusters[i,4] - self.eps) <= clusters[j, 5]
                case2 = (clusters[i,5] + self.eps) >= clusters[j, 4]
                
                if case1 and case2:
                    c1 = copy.copy(clusters[i,:])
                    c2 = copy.copy(clusters[j, :])
                    
                    merge_count+=1
                    # Expand boundaries of cluster after merging
                    if clusters[j, 4] < clusters[i, 4]:
                        clusters[i, 4] = clusters[j, 4]
                    if clusters[j, 5] > clusters[i, 5]:
                        clusters[i, 5] = clusters[j, 5]
                    clusters[i, 3] += clusters[j, 3]
                    clusters[i, 2] += clusters[j, 2]
                    if max_pt < clusters[i, 3]:
                        max_pt = clusters[i, 3]
                        max_pt_i = i
                    
                    clusters[j, 3] = 0
                    clusters[j, 4] = 21
                    clusters[j, 5] = 21
                     
                    print(f"""merging cluster [{round(c1[4],2), round(c1[5],2), round(c1[3],2)}] and [{round(c2[4],2), round(c2[5],2), round(c2[3],2)}] --> [{round(clusters[i,4],2), round(clusters[i,5], 2), round(clusters[i,3],2)}]""")
            
            self.max_pt = max_pt
            self.max_pt_i = max_pt_i
            self.merge_count = merge_count
            return clusters

    def initialize_clusters(self, max_n_clusters:int)->np.array:
        
        clusters = np.zeros((max_n_clusters, 6))
        clusters[:,4] = 21
        clusters[:,5] = 21
        
        return clusters
    
    def fitsklearn(self):
        start_idx = 0
        end_idx = start_idx + self.batch_size
        n_pad = (self.n_batches * self.batch_size) - self.z0.shape[0]
        self.z0 = self.pad_vector(self.z0, n_pad, 21)
        self.pt = self.pad_vector(self.pt, n_pad, 0)
        
        clusters_df = pd.DataFrame({})
        clusters = self.initialize_clusters(self.max_n_clusters)
        
        for i in range(self.n_batches):
            start_idx = i * self.batch_size
            end_idx = (i+1) * self.batch_size
            z0_batch = self.z0[start_idx:end_idx]
            pt_batch = self.pt[start_idx:end_idx]
            
            _db = DBSCAN(eps=0.15, min_samples=2).fit(z0_batch.reshape(-1,1))
            
            _results = pd.DataFrame({'z0':z0_batch,'pt':pt_batch,'label':_db.labels_})
            max_label = _results.label.max()
            n_noise =_results[_results.label==-1].shape[0]
            
            _results.loc[_results.label==-1, 'label'] = np.arange(n_noise)+max_label+1
            
            clusters_batch = _results.groupby(['label']).agg({'z0':[np.min, np.max],'pt':[np.sum,'count']})
            clusters_batch.columns = ['z0_min','z0_max','pt_sum','ntracks']
            clusters_df = pd.concat([clusters_df, clusters_batch])
        
        n_clusters = clusters_df.shape[0]
        clusters[0:n_clusters,2] = clusters_df['ntracks']
        clusters[0:n_clusters,3] = clusters_df['pt_sum']
        clusters[0:n_clusters,4] = clusters_df['z0_min']
        clusters[0:n_clusters,5] = clusters_df['z0_max']
        
        self.clusters_merged = self.merge_clusters(clusters)
        
        
#         self.clusters_df = clusters_df.copy()
#         clusters_raw = clusters.copy()
#         self.clusters_sklearn = pd.DataFrame(clusters_raw, columns = ['idx1','idx2','ntracks','pT','z0_low','z0_high'])
        
#         clusters_merged = self.merge_clusters(clusters)
#         self.clusters_merged_sklearn = clusters_merged
#         self.clusters_sklearn_merged = pd.DataFrame(clusters_merged, columns = ['idx1','idx2','ntracks','pT','z0_low','z0_high'])
            

    def fit(self):

        np.set_printoptions(precision=2)
        np.set_printoptions(suppress=True)

        start_idx = 0
        end_idx = start_idx + self.batch_size
        # Need to pad vectors to match the size of n_batches*batch_size
        n_pad = (self.n_batches * self.batch_size) - self.z0.shape[0]
        self.z0 = self.pad_vector(self.z0, n_pad, 21)
        self.pt = self.pad_vector(self.pt, n_pad, 0)

        # clusters = np.zeros((self.max_n_clusters, 6))
        clusters = self.initialize_clusters(self.max_n_clusters)
        # print(clusters.shape)
        # final_clusters = np.zeros((self.n_batches * self.batch_size, 6))
        # final_clusters = np.zeros((self.max_n_clusters, 6))
        pv_cluster = np.zeros((1, 6))
        merge_count = 0
        for i in range(self.n_batches):

            start_idx = i * self.batch_size
            end_idx = (i + 1) * self.batch_size

            z0_batch = self.z0[start_idx:end_idx]
            pt_batch = self.pt[start_idx:end_idx]
            # print(z0_batch.shape, pt_batch.shape)

            track_batch = self.build_tracks(z0_batch, pt_batch)

            rs_batch = self.pad_vector(
                track_batch[:, 1], self.batch_size_power_2 - self.batch_size, 0
            )

            rs_batch = np.cumsum(rs_batch)
            if self.save_intermediate:
                np.save("rs_batch.npy", rs_batch)
                np.save("pt_batch.npy", pt_batch)


            left_boundaries = self.find_left_boundaries(track_batch)
            if self.save_intermediate:
                np.save("left_boundaries_b.npy", left_boundaries)

            boundaries = self.find_right_boundaries(
                left_boundaries, rs_batch, track_batch
            )
            
            if self.save_intermediate:
                np.save("right_boundaries_b.npy", boundaries)


            clusters_batch = self.convert_boundaries_to_clusters(boundaries)

            clusters[
                i * self.max_n_clusters_batch : (i + 1) * self.max_n_clusters_batch, :
            ] = clusters_batch

            if track_batch[-1, 0] == 21:
                break

        clusters = self.merge_clusters(clusters)
        
        self.clusters = clusters

        # Find pv_cluster
        pv_cluster[0, :] = clusters[self.max_pt_i, :]

        print(self.max_pt, self.max_pt_i)
        print(f"Merged count: {self.merge_count}")

        pv_tracks = []

        for i in range(self.max_number_of_tracks):
            z0_trk = self.z0[i]

            if (z0_trk >= pv_cluster[0, 4]) and (z0_trk <= pv_cluster[0, 5]):
                pv_tracks.append(z0_trk)

        median_vertex = self.get_vertex(np.array(pv_tracks))
        self.z0_pv = np.median(pv_tracks)

        print(f"mean: {np.mean(pv_tracks)}")
        print(f"median: {np.median(pv_tracks)}")
        print(f"median2: {median_vertex}")

In [267]:
db = BatchedDBSCAN(z0, pt, eps, 130, max_number_of_tracks, True, False)

db.fitsklearn()

merging cluster [(-3.16, -1.41, 126.13)] and [(-1.76, -1.76, 13.59)] --> [(-3.16, -1.41, 139.72)]
merging cluster [(-3.16, -1.41, 139.72)] and [(-1.46, -1.46, 2.31)] --> [(-3.16, -1.41, 142.02)]
merging cluster [(-0.12, 0.41, 48.46)] and [(-0.06, -0.06, 2.99)] --> [(-0.12, 0.41, 51.44)]
merging cluster [(-3.81, -3.4, 102.77)] and [(-3.75, -3.75, 2.14)] --> [(-3.81, -3.4, 104.91)]
merging cluster [(3.57, 3.63, 10.73)] and [(3.46, 3.46, 6.31)] --> [(3.46, 3.63, 17.04)]
merging cluster [(3.87, 3.87, 2.18)] and [(3.87, 3.87, 4.25)] --> [(3.87, 3.87, 6.42)]
merging cluster [(5.27, 5.27, 2.03)] and [(5.39, 5.39, 2.25)] --> [(5.27, 5.39, 4.28)]
merging cluster [(2.46, 2.46, 3.23)] and [(2.52, 2.52, 4.19)] --> [(2.46, 2.52, 7.43)]


In [268]:
results_test = pd.DataFrame(db.clusters_merged[(db.clusters_merged[:,4]!=21) & (db.clusters_merged[:,2]>1.0)], columns = ['x','y','ntracks','pt','z0_min','z0_max'])

In [272]:
results_test.sort_values(by='pt',ascending=False)

,x,y,ntracks,pt,z0_min,z0_max
0,0.0,0.0,43.0,142.023399,-3.164062,-1.406250
5,0.0,0.0,16.0,104.907081,-3.808594,-3.398438
4,0.0,0.0,20.0,51.442479,-0.117188,0.410156
3,0.0,0.0,6.0,17.673368,-0.585938,-0.292969
10,0.0,0.0,4.0,17.044699,3.457031,3.632812
1,0.0,0.0,5.0,14.746804,2.812500,3.105469
2,0.0,0.0,5.0,13.367491,-4.218750,-4.042969
14,0.0,0.0,3.0,9.260561,0.878906,1.054688
13,0.0,0.0,3.0,8.528573,5.742188,5.859375
7,0.0,0.0,3.0,8.141763,-1.054688,-0.937500


In [220]:
db.clusters_sklearn_merged

,idx1,idx2,ntracks,pT,z0_low,z0_high
0,0.0,0.0,27.0,93.457749,-2.812500,-1.406250
1,0.0,0.0,2.0,4.330626,-4.160156,-4.042969
2,0.0,0.0,2.0,6.248494,-0.585938,-0.292969
3,0.0,0.0,2.0,6.436254,-4.218750,-4.218750
4,0.0,0.0,8.0,24.126606,-0.058594,0.410156
...,...,...,...,...,...,...
145,0.0,0.0,0.0,0.000000,21.000000,21.000000
146,0.0,0.0,0.0,0.000000,21.000000,21.000000
147,0.0,0.0,0.0,0.000000,21.000000,21.000000
148,0.0,0.0,0.0,0.000000,21.000000,21.000000


In [221]:
db.clusters_df.reset_index(drop=True, inplace=True)

In [217]:
db.clusters_df

,z0_min,z0_max,pt_sum,ntracks
label,,,,
0,-2.578125,-1.406250,93.457749,27
1,-4.042969,-4.042969,4.330626,2
2,-0.410156,-0.292969,6.248494,2
3,-4.218750,-4.218750,6.436254,2
4,0.234375,0.410156,24.126606,8
5,-3.691406,-3.398438,97.550065,14
6,-3.164062,-2.929688,14.883513,6
7,-5.625000,-5.507812,7.957812,2
8,-1.054688,-0.937500,5.098450,2


In [208]:
mask = db.clusters_sklearn['z0_low']!=21
db.clusters_sklearn.loc[mask]

,idx1,idx2,idk,pT,z0_low,z0_high
0,0.0,0.0,0.0,93.457749,-2.578125,-1.406250
1,0.0,0.0,0.0,4.330626,-4.042969,-4.042969
2,0.0,0.0,0.0,6.248494,-0.410156,-0.292969
3,0.0,0.0,0.0,6.436254,-4.218750,-4.218750
4,0.0,0.0,0.0,24.126606,0.234375,0.410156
5,0.0,0.0,0.0,97.550065,-3.691406,-3.398438
6,0.0,0.0,0.0,14.883513,-3.164062,-2.929688
7,0.0,0.0,0.0,7.957812,-5.625000,-5.507812
8,0.0,0.0,0.0,5.098450,-1.054688,-0.937500
9,0.0,0.0,0.0,4.852124,-4.921875,-4.921875


In [139]:
mask_merged = db.clusters_sklearn_merged['z0_low'] != 21
db.clusters_sklearn_merged.loc[mask_merged]

,idx1,idx2,idk,pT,z0_low,z0_high
0,0.0,0.0,0.0,93.457749,-2.812500,-1.406250
1,0.0,0.0,0.0,4.330626,-4.042969,-4.042969
2,0.0,0.0,0.0,6.248494,-0.585938,-0.292969
3,0.0,0.0,0.0,6.436254,-4.218750,-4.218750
4,0.0,0.0,0.0,24.126606,-0.058594,0.410156
5,0.0,0.0,0.0,97.550065,-3.808594,-3.398438
6,0.0,0.0,0.0,14.883513,-3.164062,-2.929688
7,0.0,0.0,0.0,7.957812,-5.625000,-5.507812
8,0.0,0.0,0.0,5.098450,-1.054688,-0.937500
9,0.0,0.0,0.0,4.852124,-4.921875,-4.921875


In [50]:
print(a)

[0.12]


In [56]:
print(" test: {}".format(a))

 test: [0.12]


In [57]:
np.printoptions(precision=2)

In [58]:
print(a[0])

0.12351235


In [83]:
clusters = pd.DataFrame(db.clusters, columns = ['idx1','idx2','idk','pT','z0_low','z0_high'])

In [88]:
clusters[clusters.z0_low!=21]

,idx1,idx2,idk,pT,z0_low,z0_high
0,0.0,0.0,0.0,5.829444,-5.625000,-5.097656
1,0.0,0.0,0.0,4.851433,-4.921875,-4.687500
2,0.0,0.0,0.0,266.443594,-4.218750,-0.761719
6,0.0,0.0,0.0,98.894986,-0.585938,2.285156
13,0.0,0.0,0.0,45.156039,2.460938,4.042969
17,0.0,0.0,0.0,18.989893,5.273438,7.031250


# Unbatched results

In [237]:
db = DBSCAN(eps=0.15, min_samples=2).fit(z0.reshape(-1,1))

In [238]:
label = db.labels_

In [238]:
label = db.labels_

In [238]:
label = db.labels_

In [239]:
results = pd.DataFrame({'z0':z0,'pt':pt,'label':label})

In [240]:
# results = results.loc[results['label']!=-1]

In [241]:
clusters = results.groupby(['label']).agg({'z0':[np.min, np.max], 'pt':[np.sum,'count']})
clusters.columns = ['z0_min','z0_max','pt_sum','ntracks']

In [242]:
clusters

,z0_min,z0_max,pt_sum,ntracks
label,,,,
-1,-9.199219,7.031250,23.986637,10
0,-3.164062,-1.406250,142.023392,43
1,2.812500,3.105469,14.746803,5
2,-4.218750,-4.042969,13.367491,5
3,-0.585938,-0.292969,17.673368,6
4,-0.117188,0.410156,51.442478,20
5,3.867188,3.867188,6.423915,2
6,-3.808594,-3.398438,104.907082,16
7,-5.625000,-5.507812,7.957811,2


In [264]:
clusters.shape[0]

21

In [152]:
z0_pv = z0[(z0>clusters['z0_min'][0]) &(z0<clusters['z0_max'][0])]

In [154]:
np.median(z0_pv)

-2.2265625

In [168]:
db.labels_.max()

19

In [172]:
db.labels_[db.labels_==-1].shape[0]

10

In [178]:
np.arange(10)+20

array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [198]:
db.loc[db.labels_==-1,'labels'] = np.arange(10)+20

AttributeError: 'DBSCAN' object has no attribute 'loc'

In [182]:
d

DBSCAN(eps=0.15, min_samples=2)

In [187]:
results.loc[results.label==-1]

,z0,pt,label


In [199]:
results.loc[results.label==-1, 'label'] = np.arange(10)+20

In [201]:
results[results.label>19]

,z0,pt,label
9,-5.097656,2.698591,20
10,5.097656,2.542250,21
30,-4.687500,1.958026,22
33,-0.761719,2.089986,23
34,7.031250,2.060123,24
61,2.285156,2.199723,25
93,-9.199219,2.812771,26
98,-6.093750,2.223069,27
107,4.042969,2.748021,28
129,-8.261719,2.654077,29
